<a href="https://colab.research.google.com/github/AxelB1011/congo/blob/master/SubredditClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install transformers
!pip install evaluate

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
train_file_path = "/content/drive/My Drive/redditComments_train.jsonlist"
test_file_path = "/content/drive/My Drive/redditComments_test_notGraded.jsonlist"


Mounted at /content/drive


In [ ]:
from datasets import Dataset, load_dataset, load_metric, ClassLabel
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
import os
import json
import evaluate

In [ ]:
label_to_int_dict = {'MarioMaker': 0, 'Nerf': 1, 'ukulele': 2, 'newToTheNavy': 3}

def json_to_pandas_train(path):
  dataset = []
  with open(path) as F:
    for l in F.readlines():
      l = json.loads(l)
      newL = dict()
      # newL['labels'] = 1 if l['submission_id'] else 0
      newL['labels'] = label_to_int_dict[l['subreddit']]
      # newL['text'] = l['submission_id'] + '[SEP]' + l['subreddit'] + '[SEP]' + l['body']
      newL['text'] = l['body']
      dataset.append(newL)
  dataset = pd.DataFrame.from_dict(dataset)
  return Dataset.from_pandas(dataset)

def json_to_pandas_test(path):
  dataset = []
  with open(path) as F:
    for l in F.readlines():
      l = json.loads(l)
      newL = dict()
      # newL['labels'] = 1 if l['submission_id'] else 0
      # newL['labels'] = l['subreddit']
      # newL['text'] = l['submission_id'] + '[SEP]' + l['subreddit'] + '[SEP]' + l['body']
      newL['text'] = l['body']
      dataset.append(newL)
  dataset = pd.DataFrame.from_dict(dataset)
  return Dataset.from_pandas(dataset)

dataset_train = json_to_pandas_train(train_file_path)
dataset_test = json_to_pandas_test(test_file_path)
#we will need it later to compute accuracy manually
dataset_test_test = json_to_pandas_train(test_file_path)

label_names = dataset_train.unique('labels') # list of labels (subreddits)
label_dict = {0: 'MarioMaker', 1: 'Nerf', 2: 'ukulele' , 3:  'newToTheNavy'}
labelss = ClassLabel(
    num_classes=len(label_names), 
    names=label_names,
    )
# for i in dataset_train['labels']:
#   label_list.append(label_dict[i])
# dataset_train['labels'] = label_list
print(dataset_train)
print(dataset_test)

Dataset({
    features: ['labels', 'text'],
    num_rows: 19600
})
Dataset({
    features: ['text'],
    num_rows: 200
})


In [ ]:
print(labelss)

ClassLabel(names=[1, 2, 3, 0], id=None)


In [ ]:
print("building tokenizer...")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(examples):
	token = tokenizer(examples["text"], padding="max_length", truncation=True)
	# token["labels"] = labelss.str2int(examples["labels"])
	# if len(examples)>1: #wont be true in case of real test dataset
	# 	token["labels"] = labelss.str2int(examples["labels"])
		# token["labels"] = [labelss.str2int(label) if label is not None else None for label in examples["labels"]]
	return token

# def tokenize_function_test(examples):
# 	return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_dataset_train = dataset_train.map(tokenize_function, batched=True)
tokenized_dataset_test = dataset_test.map(tokenize_function, batched=True)
# columns_to_return = ['input_ids', 'label', 'attention_mask']
# tokenized_dataset_test.set_format(type='torch', columns=columns_to_return)

print(tokenized_dataset_train)
print(tokenized_dataset_test)

metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
	logits,labels = eval_pred
	predictions = np.argmax(logits, axis=-1)
	result = metric.compute(predictions=predictions, references=labels)
	return result

building tokenizer...


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/043235d6088ecd3dd5fb5ca3592b6913fd516027/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.24.0",
  "vocab_size": 30522
}

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/043235d6088ecd3dd5fb5ca3592b6913fd516027/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapsh

  0%|          | 0/20 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['labels', 'text', 'input_ids', 'attention_mask'],
    num_rows: 19600
})
Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 200
})


In [ ]:
tokenized_dataset_train_split = tokenized_dataset_train.train_test_split(test_size=0.01)
tokenized_dataset_validate_split = tokenized_dataset_train_split["test"].train_test_split(test_size=0.5)

print(tokenized_dataset_train_split)
print(tokenized_dataset_validate_split)
print(tokenized_dataset_validate_split["test"]["text"])

DatasetDict({
    train: Dataset({
        features: ['labels', 'text', 'input_ids', 'attention_mask'],
        num_rows: 19404
    })
    test: Dataset({
        features: ['labels', 'text', 'input_ids', 'attention_mask'],
        num_rows: 196
    })
})
DatasetDict({
    train: Dataset({
        features: ['labels', 'text', 'input_ids', 'attention_mask'],
        num_rows: 98
    })
    test: Dataset({
        features: ['labels', 'text', 'input_ids', 'attention_mask'],
        num_rows: 98
    })
})
["World 5-3 : Boo's Mansion 2F  \nID: CHC-P3M-KJG  \nStyle: SMW Ghost House  \nThis is the third level of my Ghsot house World and it's meant to be classical Ghost House level  \n\n\nFeel free to reply with feedback and your level ID's :\\^)", 'I don’t think so. It’s probably more challenging but my philosophy is you’re better off doing something that you’re excited about because you’re much more likely to put in the time.', "Great level: lots of fun!!!\n\nHere's mine:   **5G4-MNN-8LG**"

In [ ]:
import transformers


data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="max_length")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels = len(label_names))
model.cuda()
training_args = TrainingArguments(
	output_dir="/content/drive/My Drive/ColabNotebooks/model out/",
	# optim = "adamw_torch", 
	evaluation_strategy="steps", 
	#num_train_epochs = 10,
	metric_for_best_model = "accuracy",
	# warmup_steps = 6000,
	greater_is_better = True,
	# evaluate_during_training = True, #outdated parameter option, do not use
	#no more than 30 evaluations allowed
	save_steps=6000,
	max_steps = 30000,
	eval_steps = 6000,
	per_device_train_batch_size=8, 
	per_device_eval_batch_size=8,
	weight_decay=0.01,
	# learning_rate = 2e-5,#, #defaults to 5e-5
	learning_rate = 4e-5,
	load_best_model_at_end = True,
	# lr_scheduler_type = "cosine"
	# report_to="wandb"
	)

trainer = Trainer(
		model=model,
		args=training_args,
		train_dataset=tokenized_dataset_train_split["train"],
    eval_dataset=tokenized_dataset_validate_split["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
		compute_metrics=compute_metrics
)
trainer.train()


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/043235d6088ecd3dd5fb5ca3592b6913fd516027/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.24.0",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingfac

Step,Training Loss,Validation Loss,Accuracy
6000,0.238700,0.693813,0.806122
12000,0.140900,0.868941,0.816327
18000,0.058700,1.080205,0.846939
24000,0.043600,1.381647,0.836735


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 98
  Batch size = 8
Saving model checkpoint to /content/drive/My Drive/ColabNotebooks/model out/checkpoint-6000
Configuration saved in /content/drive/My Drive/ColabNotebooks/model out/checkpoint-6000/config.json
Model weights saved in /content/drive/My Drive/ColabNotebooks/model out/checkpoint-6000/pytorch_model.bin
tokenizer config file saved in /content/drive/My Drive/ColabNotebooks/model out/checkpoint-6000/tokenizer_config.json
Special tokens file saved in /content/drive/My Drive/ColabNotebooks/model out/checkpoint-6000/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward`

Step,Training Loss,Validation Loss,Accuracy
6000,0.238700,0.693813,0.806122
12000,0.140900,0.868941,0.816327
18000,0.058700,1.080205,0.846939
24000,0.043600,1.381647,0.836735
30000,0.027700,1.489589,0.857143


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 98
  Batch size = 8
Saving model checkpoint to /content/drive/My Drive/ColabNotebooks/model out/checkpoint-30000
Configuration saved in /content/drive/My Drive/ColabNotebooks/model out/checkpoint-30000/config.json
Model weights saved in /content/drive/My Drive/ColabNotebooks/model out/checkpoint-30000/pytorch_model.bin
tokenizer config file saved in /content/drive/My Drive/ColabNotebooks/model out/checkpoint-30000/tokenizer_config.json
Special tokens file saved in /content/drive/My Drive/ColabNotebooks/model out/checkpoint-30000/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from /content/driv

TrainOutput(global_step=30000, training_loss=0.14345378850301108, metrics={'train_runtime': 3431.3713, 'train_samples_per_second': 69.943, 'train_steps_per_second': 8.743, 'total_flos': 3.1786950952157184e+16, 'train_loss': 0.14345378850301108, 'epoch': 12.37})

In [ ]:
# print(tokenized_dataset_validate_split["test"])
# # del tokenized_dataset_validate_split["test"]["labels"]
# # tokenized_dataset_validate_split["test"].drop(columns=['labels'])
# tokenized_dataset_validate_split["test"].set_format(type='torch', columns=['text', 'input_ids', 'attention_mask'])
# print(tokenized_dataset_validate_split["test"])
# # print(trainer.predict(tokenized_dataset_validate_split["test"]))

Dataset({
    features: ['labels', 'text', 'input_ids', 'attention_mask'],
    num_rows: 98
})
Dataset({
    features: ['labels', 'text', 'input_ids', 'attention_mask'],
    num_rows: 98
})


In [ ]:
import collections

# print (tokenized_dataset_test['labels'], tokenized_dataset_test['text'])
prediction = trainer.predict(tokenized_dataset_test)
print(prediction)
ans=[]
for i, elem in enumerate(prediction[0]):
  print(elem)
  temp = np.argmax(elem)
  ans.append(label_dict[temp])
print("...........")
print(ans)
# # print(prediction[0])
# srList = collections.defaultdict(set)
# for i in range(len(label_names)):
#   srList[i] = label_names[i]
# print(srList)
# ans = []
# for i in range(len(prediction[0])):
#   ans.append(srList[i])
# print("List of predicted subreddits:\n", ans)

#trainer.save_model()

# from transformers import AutoModel
# model1 = AutoModel.from_pretrained("/content/drive/My Drive/ColabNotebooks/model out/checkpoint-22000") #will be used to predict real test set
# prediction = model1.predict(tokenized_dataset_test)
# print(prediction)
# print(str(prediction[1]))

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 200
  Batch size = 8


PredictionOutput(predictions=array([[-3.403009  , -3.1340823 ,  7.478218  , -4.767856  ],
       [ 9.280364  , -3.3210082 , -3.075442  , -3.961644  ],
       [ 9.5155    , -3.453055  , -3.331469  , -3.7686446 ],
       [ 9.535079  , -3.4586804 , -3.3330088 , -3.7513022 ],
       [ 9.134587  , -3.16831   , -3.0782876 , -4.011739  ],
       [ 9.527485  , -3.4947765 , -3.3262596 , -3.7220852 ],
       [ 9.464804  , -3.4894307 , -3.199844  , -3.7753468 ],
       [ 9.530031  , -3.4548428 , -3.3649402 , -3.7413232 ],
       [ 9.49179   , -3.4421356 , -3.17396   , -3.9205806 ],
       [ 9.533851  , -3.4724863 , -3.33853   , -3.7214284 ],
       [ 9.1204    , -3.1748016 , -3.7571948 , -3.2378323 ],
       [-3.6742568 ,  0.62531924, -4.5615964 ,  5.7906466 ],
       [ 9.518833  , -3.43907   , -3.327654  , -3.763319  ],
       [ 9.53352   , -3.4692671 , -3.3575706 , -3.7138255 ],
       [ 9.531722  , -3.4747922 , -3.4008064 , -3.6829255 ],
       [ 4.5155387 , -2.151599  , -4.838363  ,  1.414227

In [ ]:
correct = 0
print(dataset_test_test)
for i in range(len(dataset_test_test)):
  print(dataset_test_test['labels'][i], label_to_int_dict[ans[i]])
  if dataset_test_test[i]['labels'] == label_to_int_dict[ans[i]]:
    correct += 1

print(f'Accuracy is {correct/len(dataset_test_test)}')


Dataset({
    features: ['labels', 'text'],
    num_rows: 200
})
0 2
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 3
0 0
0 0
0 0
0 0
0 3
0 0
0 0
0 0
0 1
0 3
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
1 0
1 1
1 1
1 1
1 1
1 3
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 2
1 1
1 2
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 0
1 1
1 1
1 1
1 1
1 3
2 2
2 2
2 3
2 2
2 2
2 2
2 2
2 2
2 2
2 1
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 3
2 2
2 2
2 2
2 2
2 1
2 1
2 2
2 2
2 0
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 0
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 1
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 1
3 3
3 3
3 3
3 3
3 3
3 3
Accuracy is 0.9
